In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import matplotlib.pyplot as plt
df=pd.read_csv('df_filtrado.csv')
# Obtener las columnas que tienen menos de 6000 NaN
columnas_sin_nans = df.columns[df.isna().sum() <= 7000]

# Crear un nuevo DataFrame filtrado
df_filtrado = df[columnas_sin_nans]

df_filtrado.head()

,Year,Seq,Disaster Group,Disaster Subgroup,Disaster Type,Disaster Subtype,Country,ISO,Region,Continent,Location,Dis Mag Scale,Start Year,Start Month,Start Day,End Year,End Month,End Day,Total Deaths,No Affected,Total Affected,CPI,Latitud,Longitud
0,1900,9002,Natural,Climatological,Drought,Drought,Cabo Verde,CPV,Western Africa,Africa,Countrywide,Km2,1900,NaN,NaN,1900,NaN,NaN,11000.0,NaN,NaN,3.221647,16.000055,-24.008395
1,1900,9001,Natural,Climatological,Drought,Drought,India,IND,Southern Asia,Asia,Bengal,Km2,1900,NaN,NaN,1900,NaN,NaN,1250000.0,NaN,NaN,3.221647,22.351115,78.667743
2,1902,12,Natural,Geophysical,Earthquake,Ground movement,Guatemala,GTM,Central America,Americas,"Quezaltenango, San Marcos",Richter,1902,4.0,18.0,1902,4.0,18.0,2000.0,NaN,NaN,3.350513,15.585555,-90.345759
3,1902,3,Natural,Geophysical,Volcanic activity,Ash fall,Guatemala,GTM,Central America,Americas,NaN,NaN,1902,4.0,8.0,1902,4.0,8.0,1000.0,NaN,NaN,3.350513,15.585555,-90.345759
4,1902,10,Natural,Geophysical,Volcanic activity,Ash fall,Guatemala,GTM,Central America,Americas,NaN,NaN,1902,10.0,24.0,1902,10.0,24.0,6000.0,NaN,NaN,3.350513,15.585555,-90.345759


In [4]:
import folium
import pandas as pd
import numpy as np
import geopandas as gpd

# Cargar los datos de los países utilizando GeoPandas
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

# Fusionar los datos de los países con los datos del DataFrame
merged_data = world.merge(df_filtrado.groupby('Country')['Total Deaths'].sum().reset_index(), left_on='name', right_on='Country', how='left')

# Aplicar una escala logarítmica a los valores de Total Deaths
merged_data['Log_Deaths'] = np.log1p(merged_data['Total Deaths'])

# Crear un objeto de mapa centrado en una ubicación específica
mapa = folium.Map(location=[-34.6037, -58.3816], zoom_start=4, min_zoom=2.5, max_bounds=True)

# Añadir los países al mapa y asignarles colores según el logaritmo de las muertes totales
folium.Choropleth(
    geo_data=merged_data,
    name='choropleth',
    data=merged_data,
    columns=['iso_a3', 'Log_Deaths'],
    key_on='feature.properties.iso_a3',
    fill_color='RdYlGn_r',  # Cambiar el color de la escala
    fill_opacity=0.7,  # Ajustar la opacidad de llenado
    line_opacity=0.2,
    line_weight=0,  # Ajustar el grosor de los bordes
    legend_name='Total Deaths (Log Scale)',
    nan_fill_color="White",
    smooth_factor=0.5
).add_to(mapa)

# Añadir un controlador de eventos de clic para mostrar el nombre del país y la cantidad de muertos
folium.GeoJson(
    merged_data,
    tooltip=folium.GeoJsonTooltip(
        fields=['name', 'Total Deaths'],
        aliases=['Country', 'Total Deaths'],
        localize=True,
        labels=True,
        sticky=True
    ),
    highlight_function=None  # Desactivar el cuadro negro al hacer clic
).add_to(mapa)

# Guardar el mapa como un archivo HTML
mapa.save('mapaa.html')


C:\Users\29196650\AppData\Local\Temp/ipykernel_20148/355143798.py:7: FutureWarning: The geopandas.dataset module is deprecated and will be removed in GeoPandas 1.0. You can get the original 'naturalearth_lowres' data from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/.
  world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
